# ЛР №10. Завдання на програмування №1

The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [7]:
import pandas as pd
import numpy as np

В цьому модулі уже є реалізована функція read_data, яка читає файл csv та форматує отриману інформацію.

In [13]:
def read_data():
    df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)
    for col in df.columns:
        if col[:2] == '01':
            df.rename(columns={col: 'Gold'+col[4:]}, inplace=True)
        elif col[:2] == '02':
            df.rename(columns={col: 'Silver'+col[4:]}, inplace=True)
        elif col[:2] == '03':
            df.rename(columns={col: 'Bronze'+col[4:]}, inplace=True)
        elif col[:1] == '№':
            df.rename(columns={col: '#'+col[1:]}, inplace=True)

    names_ids = df.index.str.split('\\s\\(') # split the index by '('

    df.index = names_ids.str[0] # the [0] element is the country name (new index)
    df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

    df = df.drop('Totals')
    #print(df)

    return df

read_data()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yugoslavia,16,26,29,28,83,14,0,3,1,4,30,26,32,29,87,YUG
Independent Olympic Participants,1,0,1,2,3,0,0,0,0,0,1,0,1,2,3,IOP
Zambia,12,0,1,1,2,0,0,0,0,0,12,0,1,1,2,ZAM
Zimbabwe,12,3,4,1,8,1,0,0,0,0,13,3,4,1,8,ZIM


### first_country(df)

Повинна повертати Series, що містить інформацію щодо першої країни в списку

In [32]:
read_data().loc["United States"]

# Summer            26
Gold               976
Silver             757
Bronze             666
Total             2399
# Winter            22
Gold.1              96
Silver.1           102
Bronze.1            84
Total.1            282
# Games             48
Gold.2            1072
Silver.2           859
Bronze.2           750
Combined total    2681
ID                 USA
Name: United States, dtype: object

In [14]:
def first_country(df):
    return df.head(1)

first_country(read_data()) 

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG


### summer_biggest(df)
повинна повертати стрічку з країною, що виграла найбільше золотих медалей в літніх іграх

In [15]:
def summer_biggest(df):
    return df.loc[df["Gold"].idxmax()].ID

summer_biggest(read_data())

'USA'

### difference_biggest(df)

повинна повертати стрічку з країною, що має найбільшу різницю (по модулю)  між кількістю золотих медалей на літніх та зимових іграх

In [16]:
def difference_biggest(df):
    return df.loc[pd.Series.abs(df["Gold"] - df["Gold.1"]).idxmax()].ID

difference_biggest(read_data())

'USA'

### difference_biggest_relative(df)
повинна повертати стрічку з країною, що має найбільшу різницю (по модулю)  між кількістю золотих медалей на літніх та зимових іграх, відносно до загальної кількості виграних золотих медалей цієї країни. Тобто, задіяна така формула:

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Включайте тільки ті країни, які виграли хоча б 1 золоту медаль влітку та взимку.

In [18]:
def difference_biggest_relative(df):
    # tmp_df = pd.concat([df.loc[(df["Gold"] > 0)&(df["Gold.1"]>0)],["Gold", "Gold.1","Gold.2" ,"ID"] ])
    tmp_df = pd.concat([df.loc[(df["Gold"] > 0)&(df["Gold.1"]>0)],["Gold", "Gold.1","Gold.2" ,"ID"] ])
    return tmp_df

difference_biggest_relative(read_data())

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

### get_points(df)

повинна додати у df стовпець Points, а також повернути тільки його (об’єкт Series). Цей стовпець буде містити кількість очок для кожної країни. За кожну золоту медаль країні присвоюється 3 бали, за кожну срібну - 2 бали і за бронзову - 1. Загальна кількість балів - сума балів.

In [13]:
def get_points(df):
    pass

get_points(df)

In [4]:

import pandas as pd

# Assuming you have a DataFrame 'a'
# Create a sample DataFrame for demonstration
data = {'Column1': [-1, 2, 0, 4],
        'Column2': [5, 1, -3, 7],
        'Column3': ['A', 'B', 'C', 'D']}
a = pd.DataFrame(data)

# Define the condition for adding a new row
new_row_data = {'Column1': 1, 'Column2': 2, 'Column3': 'E'}  # Adjust values as needed
condition = (new_row_data['Column1'] != 0) and (new_row_data['Column2'] != 0)

# Add the new row to the DataFrame only if the condition is met
result_df = a.copy()  # Create a copy of the original DataFrame
if condition:
    result_df = pd.concat([a, pd.DataFrame([new_row_data])], ignore_index=True, sort=False)

# Print the result
print(result_df)

   Column1  Column2 Column3
0       -1        5       A
1        2        1       B
2        0       -3       C
3        4        7       D
4        1        2       E
